In [ ]:
# Identify locations of deletions

chr_to_length = {'1': 249250621, '2': 243199373, '3': 198022430, '4': 191154276, '5': 180915260	CM000667.1	NC_000005.9
6	171,115,067	CM000668.1	NC_000006.11
7	159,138,663	CM000669.1	NC_000007.13
8	146,364,022	CM000670.1	NC_000008.10
9	141,213,431	CM000671.1	NC_000009.11
10	135,534,747	CM000672.1	NC_000010.10
11	135,006,516	CM000673.1	NC_000011.9
12	133,851,895	CM000674.1	NC_000012.11
13	115,169,878	CM000675.1	NC_000013.10
14	107,349,540	CM000676.1	NC_000014.8
15	102,531,392	CM000677.1	NC_000015.9
16	90,354,753	CM000678.1	NC_000016.9
17	81,195,210	CM000679.1	NC_000017.10
18	78,077,248	CM000680.1	NC_000018.9
19	59,128,983	CM000681.1	NC_000019.9
20	63,025,520	CM000682.1	NC_000020.10
21	48,129,895	CM000683.1	NC_000021.8
22	51,304,566	CM000684.1	NC_000022.10
X	155,270,560	CM000685.1	NC_000023.10
Y	59,373,566}

for i in [str(i) for i in range(1, 23)] + ['PAR1', 'PAR2']:
    deldist = np.zeros(())
    for j in range(3, 7):
        with open('sherlock_phased/chr.%s.familysize.%d.phased.txt' % (i, j), 'r')  as f:
            next(f) # skip header
            for line in f:
                pieces = line.strip().split('\t')
                family_key = pieces[0]
                inheritance_state = [int(x) for x in pieces[2:(2+(j*2))]]
                start_pos, end_pos, start_index, end_index, score = [int(x) for x in pieces[(2+(j*2)):]]
                length = end_pos - start_pos + 1
                individuals = family_to_individuals[family_key]

In [ ]:
from collections import defaultdict

# load deletions from all chromosomes
deletions = []
family_to_individuals = dict()
family_to_indices = dict()

for i in [str(i) for i in range(1, 23)] + ['PAR1', 'PAR2']:
    for j in range(3, 7):
        with open('sherlock_phased/chr.%s.familysize.%d.families.txt' % (i, j), 'r')  as f:
            next(f) # skip header
            for line in f:
                pieces = line.strip().split('\t')
                family_key = pieces[0]
                family_to_individuals[family_key] = pieces[1:(1+j)]
                family_to_indices[family_key] = [int(x) for x in pieces[(1+j):]]
                
        # load deletions
        with open('sherlock_phased/chr.%s.familysize.%d.phased.txt' % (i, j), 'r')  as f:
            next(f) # skip header
            for line in f:
                pieces = line.strip().split('\t')
                family_key = pieces[0]
                inheritance_state = [int(x) for x in pieces[2:(2+(j*2))]]
                start_pos, end_pos, start_index, end_index, score = [int(x) for x in pieces[(2+(j*2)):]]
                length = end_pos - start_pos + 1
                individuals = family_to_individuals[family_key]
                
                
                if sum(inheritance_state[:2]) > 0:
                    # maternal deletion
                    trans_op, trans_op_aut, trans_op_typ, trans, trans_aut, trans_typ = 0, 0, 0, 0, 0, 0
                    for k, child_id in enumerate(individuals[2:]):
                        is_affected = child_id_to_affected[child_id] == '2'
                        is_inherited = inheritance_state[inheritance_state[4+(2*k)]] == 1

                        trans_op += 1
                        trans_op_aut += int(is_affected)
                        trans_op_typ += int(not is_affected)
                        trans += int(is_inherited)
                        trans_aut += int(is_inherited and is_affected)
                        trans_typ += int(is_inherited and (not is_affected))
                        
                    deletions.append(Deletion(family_key, i,
                                              start_pos, end_pos, start_index, end_index, length,
                                              inheritance_state, score,
                                              True, False,
                                              trans_op, trans_op_aut, trans_op_typ,
                                              trans, trans_aut, trans_typ))
                        
                        
                if sum(inheritance_state[2:4]) > 0:
                    # paternal deletion  
                    trans_op, trans_op_aut, trans_op_typ, trans, trans_aut, trans_typ = 0, 0, 0, 0, 0, 0
                    for k, child_id in enumerate(individuals[2:]):
                        is_affected = child_id_to_affected[child_id] == '2'
                        is_inherited = inheritance_state[2+inheritance_state[5+(2*k)]] == 1

                        trans_op += 1
                        trans_op_aut += int(is_affected)
                        trans_op_typ += int(not is_affected)
                        trans += int(is_inherited)
                        trans_aut += int(is_inherited and is_affected)
                        trans_typ += int(is_inherited and (not is_affected))
                        
                    deletions.append(Deletion(family_key, i,
                                              start_pos, end_pos, start_index, end_index, length,
                                              inheritance_state, score,
                                              False, True,
                                              trans_op, trans_op_aut, trans_op_typ,
                                              trans, trans_aut, trans_typ))
                    


# remove artifact length 0 deletions
print('length 0 deletions', len([d for d in deletions if d.start_pos == d.end_pos]))
deletions = [d for d in deletions if d.start_pos != d.end_pos]

# outlier family - need to see what's going on
#outlier_families = {('AU0941', 'AU0941202', 'AU0941201'), 
#                    ('AU0951', 'AU0951202', 'AU0951201'),
#                    ('AU0453', 'AU045301', 'AU045302'),
#                    ('AU0866', 'AU0866202', 'AU0866201'),
#                    ('AU0923', 'AU0923202', 'AU0923201'),
#                    ('AU0897', 'AU0897202', 'AU0897201'),
#                    ('AU0905', 'AU0905202', 'AU0905201'),
#                    ('AU1495', 'AU1495202', 'AU1495201'),
#                    ('AU0162', 'AU016201', 'AU016202')
#                   }
#deletions = [d for d in deletions if d.family not in outlier_families]
print('deletions', len(deletions), 'maternal', len([x for x in deletions if x.is_maternal]), 'paternal', len([x for x in deletions if x.is_paternal]))

In [ ]:
import numpy as np

# position of deletions
fig, ax = plt.subplots(24, 1, sharey=True, figsize=(15, 5*22))


num_del_peaks = 0
num_recomb_peaks = 0
num_both_peaks = 0
for index, i in enumerate([str(j) for j in range(1, 23)] + ['PAR1', 'PAR2']):
    ax[index].set_title('Chrom%s' % i)
    # Deletions
    chrom_deletions = [d for d in deletions if d.chrom == i]
    if len(chrom_deletions) > 0:
        max_index = max([d.end_index for d in chrom_deletions])
        snp_positions = np.zeros((max_index+1,))
        deldist = np.zeros((max_index+1,))
        inherdist = np.zeros((max_index+1, 4), dtype=int)
        for d in chrom_deletions:
            deldist[d.start_index:(d.end_index+1)] += 1
            inherdist[d.start_index:(d.end_index+1), 0] += d.trans_aut
            inherdist[d.start_index:(d.end_index+1), 1] += (d.trans_op_aut-d.trans_aut)
            inherdist[d.start_index:(d.end_index+1), 2] += d.trans_typ
            inherdist[d.start_index:(d.end_index+1), 3] += (d.trans_op_typ-d.trans_typ)
            
            snp_positions[d.start_index] = d.start_pos
            snp_positions[d.end_index] = d.end_pos

        indices = np.where(snp_positions > 0)[0]
        #ax[index].plot(snp_positions[indices], deldist[indices])
        
        # update peaks
        num_del_peaks += np.sum((deldist>20)[1:]-(deldist>20)[:-1]>0)
        
        # cache pvalues
        trans_to_pvalue = dict([(t, chi2_contingency([[t[0], t[2]], [t[1], t[3]]])[1]) for t in set([tuple(x) for x in inherdist[deldist>20, :]])])
        
        p_values = np.ones((max_index+1,))
        p_values[deldist>20] = [trans_to_pvalue[tuple(t)] for t in inherdist[deldist>20, :]]
        ax[index].plot(snp_positions[indices], -np.log10(p_values[indices]), alpha=0.5, label='deletions')

    # Recombinations               
    chrom_recomb = [r for r in recombinations if r.chrom == i]
    if len(chrom_recomb) > 0:
        max_index = max([r.end_index for r in chrom_recomb])
        snp_positions = np.zeros((max_index+1,))
        recdist = np.zeros((max_index+1,))
        unaff_recdist = np.zeros((max_index+1,))
        aff_recdist = np.zeros((max_index+1,))
        for r in chrom_recomb:
            recdist[r.start_index:(r.end_index+1)] += 1
            if r.is_affected:
                aff_recdist[r.start_index:(r.end_index+1)] += 1
            else:
                unaff_recdist[r.start_index:(r.end_index+1)] += 1
            
            snp_positions[r.start_index] = r.start_pos
            snp_positions[r.end_index] = r.end_pos

        indices = np.where(snp_positions > 0)[0]
        
        # update peaks
        num_recomb_peaks += np.sum((recdist>20)[1:]-(recdist>20)[:-1]>0)
            
        # cache pvalues
        trans_to_pvalue = dict([((a, u), chi2_contingency([[a, u], [num_affected-a, num_unaffected-u]])[1]) for a, u in set([tuple(x) for x in zip(aff_recdist[recdist>20], unaff_recdist[recdist>20])])])

        p_values = np.ones((max_index+1,))
        p_values[recdist>20] = [trans_to_pvalue[(a, u)] for a, u in zip(aff_recdist[recdist>20], unaff_recdist[recdist>20])]
        ax[index].plot(snp_positions[indices], -np.log10(p_values[indices]), alpha=0.5, label='recombinations')
        #ax[index].plot(snp_positions[indices], \
        #               -np.log10(binom.cdf(unaff_recdist[indices], aff_recdist[indices]+unaff_recdist[indices], 1-frac_aff)), alpha=0.5, label='recombinations')
        #ax[index].plot(snp_positions[indices], recdist[indices])
        
        
    # Pull centromere positions
    with open('centromere_telomere_grch37_ucsc.txt', 'r') as f:
        for line in f:
            pieces = line.strip().split()
            if pieces[1] == 'chr%s' % i and pieces[7] == 'centromere':
                start, end = int(pieces[2]), int(pieces[3])
                ax[index].plot((start, end), (0, 0), color='purple', linewidth=10)
            if pieces[1] == 'chr%s' % i and pieces[7] == 'telomere':
                start, end = int(pieces[2]), int(pieces[3])
                ax[index].plot((start, end), (0, 0), color='red', linewidth=10)
                
    if i == 'PAR1':
        ax[index].plot(( 969238, 970836  ), (0, 0), color='orange', linewidth=10)
                
    # Pull cytogenetic coordinates
    with open('cytoBand.txt', 'r') as f:
        xticks, xticklabels = [], []
        for line in f:
            pieces = line.strip().split()
            if pieces[0] == 'chr%s' % i:
                xticks.append(int(pieces[1]))
                xticklabels.append(pieces[3])
                
    ax[index].set_xticks(xticks)
    ax[index].set_xticklabels(xticklabels, rotation='vertical')
                    
    #ax[index].set_xticks(xticks + [i for i in range(int(snp_positions[0]), int(snp_positions[-1]), 2000000)])
    #ax[index].set_xticklabels(xticklabels + [str(i/1000000) for i in range(int(snp_positions[0]), int(snp_positions[-1]), 2000000)], rotation='vertical')
                
    ax[index].axhline(y=4, linewidth=1, linestyle='--', color='r')
    ax[index].legend()
    
print('num deletion peaks', num_del_peaks)
print('num recombination peaks', num_recomb_peaks)
plt.tight_layout()
plt.show()